<a href="https://colab.research.google.com/github/aimbsg/EVA4_S13/blob/master/S13_B1_Custom_label_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
%cd '/content/gdrive/My Drive/EVA4_S13_Yolo_Custom_Dataset/'

/content/gdrive/My Drive/EVA4_S13_Yolo_Custom_Dataset


In [0]:
import pandas as pd
import json

In [0]:
json_input = json.load(open('annotation_input.json'))

In [0]:
json_input.keys()

dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])

In [0]:
json_input.values()

dict_values([{'year': 2020, 'version': '1', 'description': 'Exported using VGG Image Annotator (http://www.robots.ox.ac.uk/~vgg/software/via/)', 'contributor': '', 'url': 'http://www.robots.ox.ac.uk/~vgg/software/via/', 'date_created': 'Fri Apr 24 2020 14:19:54 GMT+0530 (India Standard Time)'}, [{'id': 0, 'width': 1920, 'height': 1080, 'file_name': 'img001.jpg', 'license': 1, 'date_captured': ''}, {'id': 1, 'width': 1920, 'height': 1080, 'file_name': 'img002.jpg', 'license': 1, 'date_captured': ''}, {'id': 2, 'width': 1920, 'height': 1200, 'file_name': 'img003.jpg', 'license': 1, 'date_captured': ''}, {'id': 3, 'width': 2880, 'height': 1800, 'file_name': 'img004.jpg', 'license': 1, 'date_captured': ''}, {'id': 4, 'width': 1920, 'height': 1200, 'file_name': 'img005.jpg', 'license': 1, 'date_captured': ''}, {'id': 5, 'width': 2560, 'height': 1600, 'file_name': 'img006.jpg', 'license': 1, 'date_captured': ''}, {'id': 6, 'width': 1920, 'height': 1080, 'file_name': 'img007.jpg', 'license': 

In [0]:
images = pd.DataFrame(json_input['images'])
annotations = pd.DataFrame(json_input['annotations'])

In [0]:
images.tail()

,id,width,height,file_name,license,date_captured
221,221,1600,1200,img222.jpg,1,
222,222,900,1600,img223.jpg,1,
223,223,736,1377,img224.jpg,1,
224,224,1280,1024,img225.jpg,1,
225,225,1920,1200,img226.jpg,1,


In [0]:
annotations.tail()

,id,image_id,category_id,segmentation,area,bbox,iscrowd
221,221,221,1,"[572, 440, 1050, 440, 1050, 1140, 572, 1140]",334600,"[572, 440, 478, 700]",0
222,222,222,1,"[69, 388, 819, 388, 819, 1169, 69, 1169]",585750,"[69, 388, 750, 781]",0
223,223,223,1,"[40, 274, 722, 274, 722, 1292, 40, 1292]",694276,"[40, 274, 682, 1018]",0
224,224,224,1,"[776, 130, 1268, 130, 1268, 568, 776, 568]",215496,"[776, 130, 492, 438]",0
225,225,225,1,"[987, 619, 1397, 619, 1397, 1156, 987, 1156]",220170,"[987, 619, 410, 537]",0


In [0]:
bb_x = []
bb_y = []
bb_x1 = []
bb_y1 = []
bb_width = []
bb_height = []
bb_list = list(annotations['bbox'])
print(len(bb_list))
print(bb_list[0])

226
[973, 33, 693, 912]


In [0]:
for i in bb_list:  
  bb_x.append(i[0])             # Existing x - top left from VGG annotator
  bb_y.append(i[1])             # Existing y - top left from VGG annotator
  bb_x1.append(i[0]+(i[2]/2))   # New x for YOLO - Image center
  bb_y1.append(i[1]+(i[3]/2))   # New y for YOLO - Image center
  bb_width.append(i[2])
  bb_height.append(i[3])
annotations['bb_x'] = bb_x
annotations['bb_y'] = bb_y
annotations['bb_x1'] = bb_x1
annotations['bb_y1'] = bb_y1
annotations['bb_width'] = bb_width
annotations['bb_height'] = bb_height  

In [0]:
annotations.head()

,id,image_id,category_id,segmentation,area,bbox,iscrowd,bb_x,bb_y,bb_x1,bb_y1,bb_width,bb_height
0,0,0,1,"[973, 33, 1666, 33, 1666, 945, 973, 945]",632016,"[973, 33, 693, 912]",0,973,33,1319.5,489.0,693,912
1,1,1,1,"[658, 103, 1556, 103, 1556, 1029, 658, 1029]",831548,"[658, 103, 898, 926]",0,658,103,1107.0,566.0,898,926
2,2,2,1,"[759, 298, 1560, 298, 1560, 982, 759, 982]",547884,"[759, 298, 801, 684]",0,759,298,1159.5,640.0,801,684
3,3,3,1,"[1492, 128, 2728, 128, 2728, 1593, 1492, 1593]",1810740,"[1492, 128, 1236, 1465]",0,1492,128,2110.0,860.5,1236,1465
4,4,4,1,"[700, 363, 1200, 363, 1200, 1184, 700, 1184]",410500,"[700, 363, 500, 821]",0,700,363,950.0,773.5,500,821


In [0]:
df_input = pd.merge(images, annotations, how='inner',on='id')

In [0]:
df_input.tail()

,id,width,height,file_name,license,date_captured,image_id,category_id,segmentation,area,bbox,iscrowd,bb_x,bb_y,bb_x1,bb_y1,bb_width,bb_height
221,221,1600,1200,img222.jpg,1,,221,1,"[572, 440, 1050, 440, 1050, 1140, 572, 1140]",334600,"[572, 440, 478, 700]",0,572,440,811.0,790.0,478,700
222,222,900,1600,img223.jpg,1,,222,1,"[69, 388, 819, 388, 819, 1169, 69, 1169]",585750,"[69, 388, 750, 781]",0,69,388,444.0,778.5,750,781
223,223,736,1377,img224.jpg,1,,223,1,"[40, 274, 722, 274, 722, 1292, 40, 1292]",694276,"[40, 274, 682, 1018]",0,40,274,381.0,783.0,682,1018
224,224,1280,1024,img225.jpg,1,,224,1,"[776, 130, 1268, 130, 1268, 568, 776, 568]",215496,"[776, 130, 492, 438]",0,776,130,1022.0,349.0,492,438
225,225,1920,1200,img226.jpg,1,,225,1,"[987, 619, 1397, 619, 1397, 1156, 987, 1156]",220170,"[987, 619, 410, 537]",0,987,619,1192.0,887.5,410,537


In [0]:
df_processed = df_input[['file_name','width','height','bb_x1','bb_y1','bb_width','bb_height']]

Scaling all 4 x,y,h,w

In [0]:
df_processed['x'] = df_processed['bb_x1'].div(df_processed['width'])
df_processed['y'] = df_processed['bb_y1'].div(df_processed['height'])
df_processed['h'] = df_processed['bb_height'].div(df_processed['height'])
df_processed['w'] = df_processed['bb_width'].div(df_processed['width'])

In [0]:
df_output = df_processed[['file_name','x','y','h','w']]
df_output['name'] = df_output.file_name.str.split('.', expand=True)[0]+'.txt'
df_output = df_output[['name','x','y','h','w']]
df_output.head() 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,x,y,h,w
0,img001.txt,0.687240,0.452778,0.844444,0.360938
1,img002.txt,0.576562,0.524074,0.857407,0.467708
2,img003.txt,0.603906,0.533333,0.570000,0.417187
3,img004.txt,0.732639,0.478056,0.813889,0.429167
4,img005.txt,0.494792,0.644583,0.684167,0.260417


In [0]:
%cd '/content/gdrive/My Drive/EVA4_S13_Yolo_Custom_Dataset/custom_data/labels/'

/content/gdrive/My Drive/EVA4_S13_Yolo_Custom_Dataset/custom_data/labels


In [0]:
for i in range(len(df_output)):
  fname = df_output.iloc[i,0]
  ftext = '0'+' '+df_output.iloc[i,1].astype(str)+' '+df_output.iloc[i,2].astype(str)+' '+df_output.iloc[i,3].astype(str)+' '+df_output.iloc[i,4].astype(str)  
  fh = open(fname, mode='w')
  fh.write(ftext)  
  fh.close()

In [0]:
df_output['name'] = df_output.name.str.split('.', expand=True)[0]+'.jpg'

In [0]:
%cd '/content/gdrive/My Drive/EVA4_S13_Yolo_Custom_Dataset/custom_data/'

/content/gdrive/My Drive/EVA4_S13_Yolo_Custom_Dataset/custom_data


In [0]:
fname = 'custom.txt'
fh = open(fname, mode='w')
fh.truncate()
for i in range(len(df_output)):  
  ftext = './data/customdata/images/'+df_output.iloc[i,0]
  fh.write(ftext)
  fh.write('\n')
fh.close()